In [5]:
from ImageProcessor import ImageProcessor
import numpy as np
import cv2
import dlib

In [6]:
# Global initialization
global_detector = dlib.get_frontal_face_detector()
global_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
global_sr_model = cv2.dnn_superres.DnnSuperResImpl_create()
global_sr_model.readModel("EDSR_x4.pb")
global_sr_model.setModel("edsr", 4)
ImageProcessor = ImageProcessor(global_detector, global_predictor, global_sr_model)

In [8]:
def get_combined_eyes(frame, global_sr_model, global_detector, global_predictor):
    """
    
    Detects and combines the eye regions from the frame using the extract_eye_region method.
    Args:
        frame: The input image frame.
        landmarks: Facial landmarks detected in the frame.
        global_sr_model: Super-resolution model.
        global_detector: Face detector.
        global_predictor: Landmark predictor.
    Returns:
        The combined eye regions, or None if not detected.
    """
    # Enhance image resolution
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = global_detector(gray)

    cv2.imshow('frame', frame)
    cv2.waitKey(0)

    for face in faces:
        landmarks = global_predictor(gray, face)

        # Extract the coordinates for each eye using dlib's facial landmark indices
        left_eye_points = range(36, 42)
        right_eye_points = range(42, 48)

        # Extract each eye region using the extract_eye_region method
        left_eye_region, _ = ImageProcessor.extract_eye_region(frame, landmarks, left_eye_points)
        right_eye_region, _ = ImageProcessor.extract_eye_region(frame, landmarks, right_eye_points)



        left_eye_super_res = ImageProcessor.enhance_image_resolution(left_eye_region, global_sr_model)
        right_eye_super_res = ImageProcessor.enhance_image_resolution(right_eye_region, global_sr_model)

        # Resize each eye region to the target size
        target_size = (40, 48)  # Assuming you want to resize both eyes to this size
        left_eye_resized = cv2.resize(left_eye_super_res, target_size)

        right_eye_resized = cv2.resize(right_eye_super_res, target_size)

        # Combine the eye regions
        combined_eyes = np.hstack((left_eye_resized, right_eye_resized))
        return combined_eyes

    return None

In [13]:
import glob
import os
import pandas as pd
local_base_dir = './data'

In [ ]:
subdirs = glob(os.path.join(local_base_dir, '*/'))

results = []

for subdir in subdirs:
    print(f"Processing images in {subdir}")
    metadata_file_path = os.path.join(subdir, 'metadata.json')
    csv_files = glob(os.path.join(subdir, '*.csv'))
    
    for csv_file in csv_files:
        dataset = pd.read_csv(csv_file, header=None)
        # Update here: Remove extra parentheses to correctly unpack arguments
        data_rows = [(tuple(row), metadata_file_path, local_base_dir) for index, row in dataset.iterrows()]

        for data_row in data_rows:
            # Update here: Unpack the arguments
            frame, metadata_file_path, local_base_dir = data_row
            combined_eyes = get_combined_eyes(frame, global_sr_model, global_detector, global_predictor)